In [1]:
import dash as ds
import dash_core_components as dcc
from jupyter_dash import JupyterDash
import dash_html_components as html
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

C:\Users\RENANL~1\AppData\Local\Temp/ipykernel_11964/1700376406.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\RENANL~1\AppData\Local\Temp/ipykernel_11964/1700376406.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
import pandas_datareader.data as web
import plotly.graph_objects as go

IBOV = web.DataReader('^BVSP', data_source='yahoo', start='2015-01-02')


In [3]:
fig_ibov = go.Figure(data=[go.Candlestick(x=IBOV.reset_index()['Date'],
                open=IBOV.reset_index()['Open'],
                high=IBOV.reset_index()['High'],
                low=IBOV.reset_index()['Low'],
                close=IBOV.reset_index()['Close'])])

## Importando dados da B3

In [4]:
def import_ibov():
    '''Funca para importar dados do site da B3 sobre a ibov, os dados no fim serão organizados em um dataframe para podermos usá-los futuramente'''
    # Importando bibliotecas
    from selenium import webdriver # Usado para importar os dados da url
    import time # usado para gerar um tempo de espera enquanto se utiliza o selenium
    from bs4 import BeautifulSoup # manipular o html
    import pandas as pd # criar a DF que será trabalhada
    
    # definindo a url da b3
    url = "https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br"
    
    # iniciando o webdriver, que deve estar com o driver na pasta do arquivo para funcionar, nesse caso será utilizado do Chrome
    browser = webdriver.Chrome()
    
    # Entrando na url informada
    browser.get(url)
    
    # Encontrando o elemento no html para filtrarmos os dados que precisamos
    browser.find_element_by_id('segment').send_keys('Setor de Atuação')
    
    # Aguardando 2 segundos para a página atualizar
    time.sleep(2)
    
    # Agora selecionando o o menu para podermos ver todas as ações disponíveis
    browser.find_element_by_id('selectPage').send_keys('120')
    
    # Aguardando 1 segundo para o página atualizar
    time.sleep(1)
    
    # IMportando o html inteiro para o Python na variável html
    html = browser.page_source
    
    # Fechando o navegador
    browser.close() 
    
    # Usando o beatiful soup e convertendo o arquivo para podermos selecionar os dados que precisamos
    bs = BeautifulSoup(html, 'html.parser')
    
    # Criando as listas onde iremos armazenar os dados do html
    setor, cod, acao, tipo, qtde_t, part, part_a = [], [], [], [], [], [], []
    
    # Usando a função find_all para selecionarmos apenas 'td' que contém os dados que usaremos e armazenando eles na variável linhas
    linhas = bs.find_all('td')
    
    # Criando uma variável que será usada no while loop abaixo
    i = 0
    
    # While loop para podermos armazenar os dados em suas respectivas colunas, onde:
    # temos 7 colunas então pegamos o primeiro elemento 0 e somando 1 até chegarmos ao último elemento para a coluna [i+6] = ao setimo elemento
    # por fim somando 7 para pularmos para a próxima linhas e seguir a lógica
    # o loop só irá para quando o número de linhas for maior do que o que os dados apresentados, para isso usados o lógica len(linhas) que nos entrega a quantia de dados
    # e como i é somado de 7 a cada iteração o loop só irá para quando todos os dados forem indexidaodos nas suas devidas colunas
    while i < (len(linhas)):
        setor.append(linhas[i].text)
        cod.append(linhas[i+1].text)
        acao.append(linhas[i+2].text)
        tipo.append(linhas[i+3].text)
        qtde_t.append(linhas[i+4].text)
        part.append(linhas[i+5].text)
        part_a.append(linhas[i+6].text)
        i += 7
        
    # Criando um df para armazearmos os dados    
    df = pd.DataFrame({'Setor': setor[:-1],
                  'Acao': cod[:-1],
                  'Empresa': acao[:-1],
                  'Qntd_teorica': qtde_t[:-1],
                  'Part_%': part[:-1]}) # filtramos para eliminar as últimas linhas, pois a b3 entrega uma soma ao final que não queremos
    # Separando os setores e subsetores em colunas
    df['SubSetor'] = df['Setor'].apply(lambda s: s[s.rfind('/')+1:].strip())
    df['Setor'] = df['Setor'].apply(lambda s: s[:s.rfind('/')])
    
    # Convertendo os valores de string para int
    df['Qntd_teorica'] = df['Qntd_teorica'].apply(lambda s: s.replace(".", ""))
    df['Qntd_teorica'] = pd.to_numeric(df['Qntd_teorica'])
    df['Part_%'] = df['Part_%'].apply(lambda s: s.replace(",", ""))
    df['Part_%'] = pd.to_numeric(df['Part_%'])/1000
    df.sort_values('Part_%', ascending = False, inplace = True)
    
    # Criando uma colunas com a % acumulada
    df['Part_%_acum'] = df['Part_%'].cumsum()
    
    # Resetando o index
    df.reset_index(drop = True, inplace = True)
    
    # fim
    return df

In [5]:
ibov = import_ibov()

## Definindo Grafico Sunburst

In [6]:
import plotly.express as px
fig  = px.sunburst(data_frame = import_ibov(), path = ['Setor', 'SubSetor', 'Empresa','Acao'], values = 'Part_%', height = 900)

In [43]:
app = JupyterDash(__name__)
app.layout = html.Div(
    children=[
        
#_________________________________________________________________________________________________________________________        
                #   Retangulo de entrada     #
                html.Div(
                    children=[
                    html.H2(
                    children='',
                    style={
                        'color': 'black',
                        'width': '100px',
                        'height': '100px',
                        'border-radius': '12px',
                    }
                ),

                html.H1(
                    children="📈 Ações IBOV ", className="header-title",
                    style={'color': '#CFCFCF',
                            'margin': '4px auto',
                            'max-width': '384px',
                            'text-align':'center',
                            'display':'inline-block',
                            'display': 'flex'
                            'font: sans-serif'
                            }
                ),
                
                
            ], style={'background-color':'rgb(15, 15, 15)', 'border-radius': '5px'},
            className="header",
        ),
        
#__________________________________________________________________________________________________________________________        
                dcc.Graph(
                    figure = px.sunburst(data_frame = ibov, path = ['Setor', 'SubSetor', 'Empresa','Acao'], values = 'Part_%'),
                    style={'display' : 'inline-block', 
                            'height': '100vh'
                          },
                ),
#__________________________________________________________________________________________________________________________        

        
#__________________________________________________________________________________________________________________________                
                dcc.Graph(
                    figure=go.Figure(data=[go.Candlestick(x=IBOV.reset_index()['Date'],
                        open=IBOV.reset_index()['Open'],
                        high=IBOV.reset_index()['High'],
                        low=IBOV.reset_index()['Low'],
                        close=IBOV.reset_index()['Close'])]),
                    style={'display':'inline-block', 'width': '100vh', 'height': '80vh'},
                )]
)

In [44]:
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [ ]:
app.run_server(mode='inline')

In [ ]:
import pandas_datareader.data as web
import pandas as pd
acoes = ['ELET3.SA', 'EQTL3.SA', 'TAEE11.SA', 'ENGI11.SA', 'CPLE6.SA', 'CMIG4.SA', 'EGIE3.SA', 
         'ENEV3.SA', 'ENBR3.SA', 'CPFE3.SA', '^BVSP']

acoes_df = pd.DataFrame()
for acao in acoes:
    acoes_df[acao] = web.DataReader(acao, data_source='yahoo', start='2021-01-01')['Close']
acoes_df.reset_index(inplace = True)    

In [ ]:
#1. ELETROBRAS
#2. EQUATORIAL
#3. TAESA
#4. ENERGISA
#5. COPEL
#6. CEMIG
#7. ENGIE
#8. ENEVA
#9. ENERGIAS DO BRASIL
#10. CPFL ENERGIA

In [ ]:
import dash
import dash_html_components as html
import plotly.graph_objects as go
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output
from datetime import date, datetime

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.H1(id = 'H1', children = 'Principais Empresas ⚡', style = {'textAlign':'center',\
                                                'marginTop':40,'marginBottom':40}),                          
        html.Div([
            
            html.Label('Data'),
            dcc.DatePickerRange(
                initial_visible_month=datetime.now(),
                start_date=date(2021, 7, 1),
                end_date=date(2021, 7, 14), style = {'display' : 'inline', 'width' : '50vh', 'height' : '50vh'}),
            html.Br(),
            html.Label('Acao'),                      
            dcc.Dropdown( id = 'dropdown',
                options = [
                    {'label': 'ELETROBRAS', 'value':acoes[0] },
                    {'label': 'EQUATORIAL', 'value':acoes[1]},
                    {'label': 'TAESA', 'value':acoes[2]},
                    {'label': 'ENERGISA', 'value':acoes[3]},
                    {'label': 'COPEL', 'value':acoes[4]},
                    {'label': 'CEMIG', 'value':acoes[5]},
                    {'label': 'ENGIE', 'value':acoes[6]},
                    {'label': 'ENEVA', 'value':acoes[7]},
                    {'label': 'ENERGIAS DO BRASIL', 'value':acoes[8]},
                    {'label': 'CPFL ENERGIA', 'value':acoes[9]},            
                        ],
                value = acoes[0], style = {'width' : '50vh', 'display':'d'})
        
                ],)
                ], style={'columnCount': 2}),
                                   
                          
        dcc.Graph(id = 'bar_plot')
    ])
    
    
@app.callback(Output(component_id='bar_plot', component_property= 'figure'),
              [Input(component_id='dropdown', component_property= 'value')])
def graph_update(dropdown_value):
    fig = go.Figure([go.Scatter(x = acoes_df['Date'], y = acoes_df['{}'.format(dropdown_value)],)
                     ])
    
    fig.update_layout(title = 'Stock prices over time',
                      xaxis_title = 'Dates',
                      yaxis_title = 'Prices'
                      )
    return fig  



if __name__ == "__main__":
    app.run_server(debug=True)

In [ ]:
for i in acoes:
    list(i)
    

In [ ]:
acoes[0]

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Label('Dropdown'),
    dcc.Dropdown(
        options=[
            {'label': 'New York City', 'value': 'NYC'},
            {'label': u'Montréal', 'value': 'MTL'},
            {'label': 'San Francisco', 'value': 'SF'}
        ],
        value='MTL'
    ),

    html.Label('Multi-Select Dropdown'),
    dcc.Dropdown(
        options=[
            {'label': 'New York City', 'value': 'NYC'},
            {'label': u'Montréal', 'value': 'MTL'},
            {'label': 'San Francisco', 'value': 'SF'}
        ],
        value=['MTL', 'SF'],
        multi=True
    ),

    html.Label('Radio Items'),
    dcc.RadioItems(
        options=[
            {'label': 'New York City', 'value': 'NYC'},
            {'label': u'Montréal', 'value': 'MTL'},
            {'label': 'San Francisco', 'value': 'SF'}
        ],
        value='MTL'
    ),

    html.Label('Checkboxes'),
    dcc.Checklist(
        options=[
            {'label': 'New York City', 'value': 'NYC'},
            {'label': u'Montréal', 'value': 'MTL'},
            {'label': 'San Francisco', 'value': 'SF'}
        ],
        value=['MTL', 'SF']
    ),

    html.Label('Text Input'),
    dcc.Input(value='MTL', type='text'),

    html.Label('Slider'),
    dcc.Slider(
        min=0,
        max=9,
        marks={i: 'Label {}'.format(i) if i == 1 else str(i) for i in range(1, 6)},
        value=5,
    ),
], style={'columnCount': 2})

app.run_server(mode='inline')